<a href="https://colab.research.google.com/github/83tu1/83tu1/blob/main/Route_Invites_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Connections for BQ & Postgres

In [13]:
# Install all necessary packages
!pip install ortools -q

!pip install google-cloud-bigquery
!pip install psycopg2-binary


In [14]:
# Import all necessary packages
from google.colab import userdata
from sqlalchemy import create_engine
import pandas as pd
from google.cloud import bigquery
import numpy as np

#Authenticate to use google services
from google.colab import auth
auth.authenticate_user()

from ortools.linear_solver import pywraplp
from google.colab import files


In [15]:
# Big Query Connection Settings

# Set your project ID
project_id = 'relaytech-production'

# Create a BigQuery client with the project ID
client = bigquery.Client(project=project_id)

### Connect to postgres via google cloud SDK

In [16]:
!curl https://sdk.cloud.google.com | bash # Download the Google Cloud SDK installation script

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   443  100   443    0     0   5382      0 --:--:-- --:--:-- --:--:--  5402
############################################################################################# 100.0%
Running install script from: /tmp/tmp.E7XKYnpM12/install_google_cloud_sdk.bash
which curl
curl -# -f https://dl.google.com/dl/cloudsdk/channels/rapid/google-cloud-sdk.tar.gz
############################################################################################# 100.0%

mkdir -p /root
"/root/google-cloud-sdk" already exists and may contain out of date files.
Remove /root/google-cloud-sdk or select a new installation directory, then run again.


In [17]:
# After executing this command, all subsequent google cloud commands will operate within the context of the relaytech-production project
!gcloud config set project relaytech-production

Updated property [core/project].


In [18]:
# The first command downloads the Cloud SQL Proxy binary, which is a lightweight application that allows you to connect to your
#  Google Cloud SQL databases from your local environment or other services securely.
!curl -o cloud_sql_proxy https://dl.google.com/cloudsql/cloud_sql_proxy.linux.amd64

# The second command changes the permissions of the downloaded file to make it executable. This is necessary because, by default,
# files downloaded from the internet may not have the appropriate permissions to be run directly.
!chmod +x cloud_sql_proxy


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19.0M  100 19.0M    0     0  22.4M      0 --:--:-- --:--:-- --:--:-- 22.4M


In [19]:
# Runs the Cloud SQL Proxy in the Background: The command starts the Cloud SQL Proxy, allowing local applications to connect to the
# specified Google Cloud SQL instance over TCP on port 3391.

!nohup ./cloud_sql_proxy -instances=relaytech-production:europe-west1:relayapi-clone-analytics=tcp:3391 > cloud_sql_proxy.log 2>&1 &


In [20]:
# Display recent log entries to monitor the cloud SQL proxy

!tail -n 10 cloud_sql_proxy.log


2024/11/19 11:23:03 This is the Cloud SQL Proxy v1. It is no longer receiving active feature development. For the latest features and improvements, migrate to the v2 version of the Cloud SQL Proxy. For details, see: https://github.com/GoogleCloudPlatform/cloud-sql-proxy/blob/main/migration-guide.md
2024/11/19 11:23:03 current FDs rlimit set to 1048576, wanted limit is 8500. Nothing to do here.


In [21]:
host = "localhost"
port = "3391"
database = "relayapi"
user = "admin"
password = userdata.get('password')

# Create a SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

del password

## Query & Data Manipulation

### Big Query Query
- courier-route spine incl. relevancy, latest location, start location & most recent worked date


In [22]:
# Get a list of all possible courier / route combinations, including stats on the couriers, such as latest lat long and start location. This query also filters to ensure that only rows exist for couriers and routes in the same uk region (london & NW)
courier_route_spine = """
with most_recent_lat_lon as (
    select
    user_id
  , lat
  , lon
  , event_date
  , event_timestamp
from copilot.copilot_events
where 1=1
and lat is not null
and lon is not null
qualify row_number() over(partition by user_id order by event_timestamp desc) = 1
)

, start_lat_lon as (
    select
    user_id
  , start_point_lat
  , start_point_lon
  , event_date
  , event_timestamp
from copilot.copilot_events
where 1=1
and start_point_lat is not null
and start_point_lon is not null
qualify row_number() over(partition by user_id order by event_timestamp desc) = 1
)

, get_couriers_worked_last_x_days as (
                select cj.courier_uid
                    , cj.cps_tier
                    , mr.lat as most_recent_lat
                    , mr.lon as most_recent_lon
                    , mr.event_timestamp as most_recent_lat_lon_timestamp
                    , cj.routes__most_recent_route_on
                    , sl.start_point_lat
                    , sl.start_point_lon
                    , case when cj.assigned_location = "manchester" then "North West" else "London" end as courier_region
                 from reports.courier_journeys cj
                 left join most_recent_lat_lon mr on mr.user_id = cj.ga_user_id
                 left join start_lat_lon sl on sl.user_id = cj.ga_user_id
                 where cj.routes__most_recent_route_on > CURRENT_DATE() - 14)


, route_courier_cross_join as (
select rp.route_uid
                , c.courier_uid
                , c.cps_tier
                , c.most_recent_lat
                , c.most_recent_lon
                , c.most_recent_lat_lon_timestamp
                , c.start_point_lat
                , c.start_point_lon
                , c.routes__most_recent_route_on
                , o.UK_region
                , c.courier_region
           from reports.routes_performance rp
            left join utilities.uk_outcode o on o.outcode = rp.outcode_distribution__primary_outcode
                    cross join get_couriers_worked_last_x_days c
           where rp.route_date = CURRENT_DATE()
             and rp.state <> "cancelled"
             and UK_region = courier_region --Only include courier route combinations in the same national region

)

select rc.*
    , rr.primary_outcode__all_time_shipments
    , rr.scenario_super_relevant.ratio_relevant_drops_to_route_shipments as super_relevant_drops_ratio
    , rr.scenario_relevant.ratio_relevant_drops_to_route_shipments as relevant_drops_ratio
from route_courier_cross_join rc
left join staging.stg_route_relevancy rr on rr.route_uid = rc.route_uid and rr.courier_uid = rc.courier_uid
order by rc.route_uid


"""

all_courier_route_combinations = client.query(courier_route_spine).to_dataframe()

all_courier_route_combinations.info()
all_courier_route_combinations.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134696 entries, 0 to 134695
Data columns (total 14 columns):
 #   Column                               Non-Null Count   Dtype              
---  ------                               --------------   -----              
 0   route_uid                            134696 non-null  object             
 1   courier_uid                          134696 non-null  object             
 2   cps_tier                             109866 non-null  object             
 3   most_recent_lat                      134368 non-null  float64            
 4   most_recent_lon                      134368 non-null  float64            
 5   most_recent_lat_lon_timestamp        134368 non-null  datetime64[us, UTC]
 6   start_point_lat                      116265 non-null  float64            
 7   start_point_lon                      116265 non-null  float64            
 8   routes__most_recent_route_on         134696 non-null  dbdate             
 9   UK_region      

,route_uid,courier_uid,cps_tier,most_recent_lat,most_recent_lon,most_recent_lat_lon_timestamp,start_point_lat,start_point_lon,routes__most_recent_route_on,UK_region,courier_region,primary_outcode__all_time_shipments,super_relevant_drops_ratio,relevant_drops_ratio
0,006208e8-4711-4f5b-b562-1a739ba7846d,edc805d1-f136-46bc-b2a3-c466dc5c0429,None,53.376480,-2.101918,2024-11-19 10:55:10.902507+00:00,53.459771,-2.060431,2024-11-19,North West,North West,<NA>,NaN,NaN
1,006208e8-4711-4f5b-b562-1a739ba7846d,46e124d1-c174-499b-847e-5ca82e8f129d,GOLD,53.498346,-2.640060,2024-11-18 15:47:39.410000+00:00,53.514572,-2.152800,2024-11-18,North West,North West,<NA>,NaN,NaN
2,006208e8-4711-4f5b-b562-1a739ba7846d,d21bd741-6b09-4103-8101-56ad71275bfb,None,53.408040,-2.988525,2024-11-18 13:51:55.430000+00:00,53.415597,-3.001313,2024-11-18,North West,North West,<NA>,NaN,NaN
3,006208e8-4711-4f5b-b562-1a739ba7846d,87edcd86-1883-4038-8bad-709fab1dc16a,None,53.382167,-2.148115,2024-11-18 19:08:20.911039+00:00,NaN,NaN,2024-11-18,North West,North West,<NA>,NaN,NaN
4,006208e8-4711-4f5b-b562-1a739ba7846d,cee80a96-2ecb-4901-86f5-54ac5582b6fa,PLATINUM,53.522070,-2.397967,2024-11-11 20:44:23.850008+00:00,53.431312,-2.964987,2024-11-11,North West,North West,<NA>,NaN,NaN


### Postgres

- Open Routes
- Courier availability & suspension status

In [23]:
# Get open routes and pickup location
open_routes = pd.read_sql(
"""

select r.uid as route_uid
    , p.geo_lat as pitstop_lat
    , p.geo_long as pitstop_lon
    , true as open_route_flag

from routes r
left join rt_pitstops p on r.collection_pitstop_uid = p.uid
where starts_on = current_date
and courier_uid is null
and state <> 'cancelled'

;""", engine)

#Format route_uid as a string and remove whitespaces
open_routes['route_uid'] = open_routes['route_uid'].astype(str).str.strip()

# Display the DataFrame
open_routes.info()
open_routes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   route_uid        52 non-null     object 
 1   pitstop_lat      52 non-null     float64
 2   pitstop_lon      52 non-null     float64
 3   open_route_flag  52 non-null     bool   
dtypes: bool(1), float64(2), object(1)
memory usage: 1.4+ KB


,route_uid,pitstop_lat,pitstop_lon,open_route_flag
0,51ca9563-ad99-43f1-9884-b5cb828a933e,51.490005,-0.228736,True
1,fea6ccbd-73d4-460e-8fc4-35bd9a0334b0,51.485199,0.102190,True
2,eeba5973-62ec-4073-b044-8406786f938b,51.432297,-0.021368,True
3,fad704a3-6b53-4c3d-8ad4-098aa1cef814,51.442374,-0.153356,True
4,db9ea561-a02f-49cc-9101-b33af4a14cac,51.470325,-0.139066,True


In [24]:
# Get courier availability and suspension status
courier_status = pd.read_sql(
"""
with suspended_couriers as (
    select courier_uid
        , t.type
        , t.value
        , true as suspended_flag
from courier_tags as ct
left join tags t on t.uid = ct.tag_uid
where current_timestamp between ct.valid_from and ct.valid_to
and t.value in ('bookings_suspended')
)

, fully_booked_couriers as (
    select courier_uid
        , t.type
        , t.value
        , true as fully_booked_flag
from courier_tags as ct
left join tags t on t.uid = ct.tag_uid
where current_timestamp between ct.valid_from and ct.valid_to
and t.type in ('availability')
and t.value in ('none')
)

select distinct
        ct.courier_uid
        , suspended_flag
        , fully_booked_flag
from courier_tags as ct
left join suspended_couriers sc on sc.courier_uid = ct.courier_uid
left join fully_booked_couriers fbc on fbc.courier_uid = ct.courier_uid
where  not(suspended_flag is null and fully_booked_flag is null)
order by suspended_flag asc
;""", engine)

#Format route_uid as a string and remove whitespaces
courier_status['courier_uid'] = courier_status['courier_uid'].astype(str).str.strip()

# Display the DataFrame
courier_status.info()
courier_status.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   courier_uid        62 non-null     object
 1   suspended_flag     49 non-null     object
 2   fully_booked_flag  13 non-null     object
dtypes: object(3)
memory usage: 1.6+ KB


,courier_uid,suspended_flag,fully_booked_flag
0,86b256df-40f5-4a91-85d2-016ce515fba6,True,None
1,1a049f15-0b02-4f67-b4db-ed0dad9da277,True,None
2,7ff16d85-b725-4572-96c6-7f51d48d4635,True,None
3,02eebe12-3a58-4ac5-9058-5c5a8e719369,True,None
4,84d34694-b1d1-4f31-8272-b67b30fc607c,True,None


### Join Dataframes

In [25]:
# Join the 3 dataframes
courier_route_join_open_routes = pd.merge(all_courier_route_combinations, open_routes, on='route_uid', how='left')
courier_route_join_courier_status = pd.merge(courier_route_join_open_routes, courier_status, on='courier_uid', how='left')
# courier_route_join_courier_status.head()

#Fill flag blanks with false
courier_route_join_courier_status['open_route_flag'] = courier_route_join_courier_status['open_route_flag'].fillna(False)
courier_route_join_courier_status['suspended_flag'] = courier_route_join_courier_status['suspended_flag'].fillna(False)
courier_route_join_courier_status['fully_booked_flag'] = courier_route_join_courier_status['fully_booked_flag'].fillna(False)

#Filter to only open routes
open_courier_route_combinations = courier_route_join_courier_status[courier_route_join_courier_status['open_route_flag'] == True]

# Filter to only non-suspended and available couriers
open_courier_route_combinations = open_courier_route_combinations[open_courier_route_combinations['suspended_flag'] == False]
open_courier_route_combinations = open_courier_route_combinations[open_courier_route_combinations['fully_booked_flag'] == False]

# Take last known lat long only if its today, otherwise take start lat lon
from datetime import date
today = date.today()
open_courier_route_combinations['lat'] = np.where(
    pd.to_datetime(open_courier_route_combinations['most_recent_lat_lon_timestamp']).dt.date == today,
    open_courier_route_combinations['most_recent_lat'],
    open_courier_route_combinations['start_point_lat']
)
open_courier_route_combinations['lon'] = np.where(
    pd.to_datetime(open_courier_route_combinations['most_recent_lat_lon_timestamp']).dt.date == today,
    open_courier_route_combinations['most_recent_lon'],
    open_courier_route_combinations['start_point_lon']
)
# If there is no start_point then take the most recent location
open_courier_route_combinations['lat'] = np.where(
   open_courier_route_combinations['start_point_lat'].isnull(),
    open_courier_route_combinations['most_recent_lat'],
    open_courier_route_combinations['lat']
)

open_courier_route_combinations['lon'] = np.where(
    open_courier_route_combinations['start_point_lon'].isnull(),
    open_courier_route_combinations['most_recent_lon'],
    open_courier_route_combinations['lon']
)

open_courier_route_combinations['location_source'] = np.where(
    open_courier_route_combinations['lat'] == open_courier_route_combinations['most_recent_lat'],
    'most_recent_location',
    'start_point'
)
open_courier_route_combinations.info()
open_courier_route_combinations.head()


<ipython-input-25-4b27a8c86d0a>:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  courier_route_join_courier_status['open_route_flag'] = courier_route_join_courier_status['open_route_flag'].fillna(False)
<ipython-input-25-4b27a8c86d0a>:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  courier_route_join_courier_status['suspended_flag'] = courier_route_join_courier_status['suspended_flag'].fillna(False)
<ipython-input-25-4b27a8c86d0a>:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future versi

<class 'pandas.core.frame.DataFrame'>
Index: 12727 entries, 2425 to 134401
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype              
---  ------                               --------------  -----              
 0   route_uid                            12727 non-null  object             
 1   courier_uid                          12727 non-null  object             
 2   cps_tier                             10881 non-null  object             
 3   most_recent_lat                      12688 non-null  float64            
 4   most_recent_lon                      12688 non-null  float64            
 5   most_recent_lat_lon_timestamp        12688 non-null  datetime64[us, UTC]
 6   start_point_lat                      10842 non-null  float64            
 7   start_point_lon                      10842 non-null  float64            
 8   routes__most_recent_route_on         12727 non-null  dbdate             
 9   UK_region                    

,route_uid,courier_uid,cps_tier,most_recent_lat,most_recent_lon,most_recent_lat_lon_timestamp,start_point_lat,start_point_lon,routes__most_recent_route_on,UK_region,...,super_relevant_drops_ratio,relevant_drops_ratio,pitstop_lat,pitstop_lon,open_route_flag,suspended_flag,fully_booked_flag,lat,lon,location_source
2425,04830d1d-51d2-4ef7-9883-26b9f70f3f65,f22e72a3-0b5e-4eb2-a13b-e710950bf835,GOLD,53.613484,-2.488571,2024-11-18 16:57:53.321004+00:00,53.521576,-2.397439,2024-11-18,North West,...,NaN,NaN,53.580832,-2.395678,True,False,False,53.521576,-2.397439,start_point
2426,04830d1d-51d2-4ef7-9883-26b9f70f3f65,ca3efb2f-db36-42c0-82f7-147d82bdb3fe,PLATINUM,53.588943,-2.227087,2024-11-16 18:29:19.001048+00:00,53.536889,-2.381906,2024-11-16,North West,...,NaN,NaN,53.580832,-2.395678,True,False,False,53.536889,-2.381906,start_point
2427,04830d1d-51d2-4ef7-9883-26b9f70f3f65,ea83954b-95af-4722-95bb-f5cde420b0e7,BRONZE,53.577452,-2.206957,2024-11-18 21:47:49.294054+00:00,53.616752,-2.128771,2024-11-18,North West,...,NaN,NaN,53.580832,-2.395678,True,False,False,53.616752,-2.128771,start_point
2428,04830d1d-51d2-4ef7-9883-26b9f70f3f65,b51b940e-2829-476c-874c-41f3fba4b74d,SILVER,53.488315,-2.356356,2024-11-18 20:01:52.170001+00:00,NaN,NaN,2024-11-19,North West,...,NaN,NaN,53.580832,-2.395678,True,False,False,53.488315,-2.356356,most_recent_location
2429,04830d1d-51d2-4ef7-9883-26b9f70f3f65,3e639386-a5f0-4336-91be-76adc18f9f67,PLATINUM,53.479943,-2.258694,2024-11-14 22:14:30.321005+00:00,53.476861,-2.263211,2024-11-14,North West,...,NaN,NaN,53.580832,-2.395678,True,False,False,53.476861,-2.263211,start_point


In [26]:
# Calculate distance from location to pitstop pickup
import math

# Define function
def haversine_distance(row):
    # Extract coordinates from the row
    lat1 = row['lat']
    lon1 = row['lon']
    lat2 = row['pitstop_lat']
    lon2 = row['pitstop_lon']

    # Radius of Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    # Haversine formula
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distance in kilometers
    distance = R * c
    return distance

# Apply function
open_courier_route_combinations['distance_to_pitstop'] = open_courier_route_combinations.apply(haversine_distance, axis=1)
open_courier_route_combinations.head()

,route_uid,courier_uid,cps_tier,most_recent_lat,most_recent_lon,most_recent_lat_lon_timestamp,start_point_lat,start_point_lon,routes__most_recent_route_on,UK_region,...,relevant_drops_ratio,pitstop_lat,pitstop_lon,open_route_flag,suspended_flag,fully_booked_flag,lat,lon,location_source,distance_to_pitstop
2425,04830d1d-51d2-4ef7-9883-26b9f70f3f65,f22e72a3-0b5e-4eb2-a13b-e710950bf835,GOLD,53.613484,-2.488571,2024-11-18 16:57:53.321004+00:00,53.521576,-2.397439,2024-11-18,North West,...,NaN,53.580832,-2.395678,True,False,False,53.521576,-2.397439,start_point,6.589953
2426,04830d1d-51d2-4ef7-9883-26b9f70f3f65,ca3efb2f-db36-42c0-82f7-147d82bdb3fe,PLATINUM,53.588943,-2.227087,2024-11-16 18:29:19.001048+00:00,53.536889,-2.381906,2024-11-16,North West,...,NaN,53.580832,-2.395678,True,False,False,53.536889,-2.381906,start_point,4.970153
2427,04830d1d-51d2-4ef7-9883-26b9f70f3f65,ea83954b-95af-4722-95bb-f5cde420b0e7,BRONZE,53.577452,-2.206957,2024-11-18 21:47:49.294054+00:00,53.616752,-2.128771,2024-11-18,North West,...,NaN,53.580832,-2.395678,True,False,False,53.616752,-2.128771,start_point,18.059593
2428,04830d1d-51d2-4ef7-9883-26b9f70f3f65,b51b940e-2829-476c-874c-41f3fba4b74d,SILVER,53.488315,-2.356356,2024-11-18 20:01:52.170001+00:00,NaN,NaN,2024-11-19,North West,...,NaN,53.580832,-2.395678,True,False,False,53.488315,-2.356356,most_recent_location,10.610584
2429,04830d1d-51d2-4ef7-9883-26b9f70f3f65,3e639386-a5f0-4336-91be-76adc18f9f67,PLATINUM,53.479943,-2.258694,2024-11-14 22:14:30.321005+00:00,53.476861,-2.263211,2024-11-14,North West,...,NaN,53.580832,-2.395678,True,False,False,53.476861,-2.263211,start_point,14.502304


In [27]:
# Calculate the relevancy score which is based on how many drops for each route are done in super liked and liked outcodes
open_courier_route_combinations['preference_relevancy_score'] = open_courier_route_combinations['super_relevant_drops_ratio'] + open_courier_route_combinations['relevant_drops_ratio']*0.5
open_courier_route_combinations['preference_relevancy_score'] = open_courier_route_combinations['preference_relevancy_score'].fillna(0)

# Calculate Proximity score
open_courier_route_combinations['proximity_score'] = 2 * np.exp(-0.25 * open_courier_route_combinations['distance_to_pitstop'])
open_courier_route_combinations['proximity_score'] = open_courier_route_combinations['proximity_score'].fillna(0)

# Add scores to create invite_score
open_courier_route_combinations['invite_score'] = open_courier_route_combinations['preference_relevancy_score'] + open_courier_route_combinations['proximity_score']


[link text](https://)## Initiate Solver & Download Results

### Set up preference matrix and worst route vector for optimisation

In [28]:
# Pivot the DataFrame to create the matrix
pivot_df = open_courier_route_combinations.pivot(index='courier_uid', columns='route_uid', values='invite_score')

# Fill missing values with 0 (or another appropriate value)
pivot_df = pivot_df.fillna(0)

# Convert the DataFrame to a 2D NumPy array (matrix P)
preference_matrix = pivot_df.to_numpy()

# Get the list of workers and shifts (optional)
couriers = pivot_df.index.tolist()
routes = pivot_df.columns.tolist()

# Display the matrix
print("preference_matrix:")
print(preference_matrix)

# Optional: Display the workers and shifts
print("Couriers:", couriers)
print("Routes:", routes)

# Calculate the Route importance vector. This is the normalised inverse of the sum of all preference scores
sum_scores_per_route = preference_matrix.sum(axis=0)
print("sum_scores:", sum_scores_per_route)

# Step 1: Inverse the values
inverse_sum_scores_per_route = np.max(sum_scores_per_route) - sum_scores_per_route

# Step 2: Min-Max Normalize the inverted values
min_score = np.min(inverse_sum_scores_per_route)
max_score = np.max(inverse_sum_scores_per_route)

# Step 3: Calculate fill_difficulty_score it is between 1 and 2. ie. Worst route is twice as important as best route
fill_difficulty_score = 1 + (inverse_sum_scores_per_route - min_score) / (max_score - min_score)
fill_difficulty_score = np.nan_to_num(fill_difficulty_score,nan=1)
print("Inverse and Min-Max Normalized D:", fill_difficulty_score)

preference_matrix:
[[0.00000000e+00 1.55455259e-02 7.00468946e-03 ... 9.30878586e-02
  1.10512242e-01 5.76641315e-02]
 [0.00000000e+00 5.97178538e-01 1.21927730e+00 ... 5.36449342e-02
  3.22324514e-02 1.11524985e-01]
 [0.00000000e+00 7.31456482e-02 4.29509828e-02 ... 5.26375241e-01
  7.74141857e-02 4.86015632e-01]
 ...
 [7.24189802e-02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.02409784e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.57761045e-03 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
Couriers: ['00b08e94-94fe-42bb-84da-4c25360b24ea', '016b586e-a665-4eaa-b38c-bae8c5b954a4', '020b48fc-84b2-480e-9418-ab1720d7e913', '023cf413-a6b6-4c54-a6fd-7907ba0c933f', '024523d0-fbb9-4d7e-830e-5cbab58b555e', '029677f0-07a6-40d4-ac60-8a2f0c8a7b06', '032ad2b4-be0e-4cf9-9a6c-651262e1d1eb', '034f6979-952d-4866-8fa1-a7571f01b216', '03be0e5f-f7c0-4768-ade9-cd6582eb486a', '061988f5-

# Initiate the solver and output as csv

In [29]:

def maximize_shift_filling(couriers, routes, preference_matrix, route_importance_vector):
    # Initialize the solver
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Number of couriers and routes
    n = len(couriers)
    m = len(routes)

    # Decision variables: x[i][j] = 1 if courier i is invited to route j, else 0
    x = {}
    for i in range(n):
        for j in range(m):
            x[i, j] = solver.IntVar(0, 1, f'x[{i},{j}]')

    # Constraints
    # 1. Each route can have at most 5 invites
    for j in range(m):
        solver.Add(sum(x[i, j] for i in range(n)) <= 5)

    # 2. Each courier can receive at most 1 invites
    for i in range(n):
        solver.Add(sum(x[i, j] for j in range(m)) <= 1)

    # Objective function: Maximize total likelihood of filling routes
    objective = solver.Objective()
    for i in range(n):
        for j in range(m):
            objective.SetCoefficient(x[i, j], preference_matrix[i][j] * fill_difficulty_score[j])
    objective.SetMaximization()

    # Solve the problem
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution found!')
        solution = {}
        for i in range(n):
            for j in range(m):
                if x[i, j].solution_value() > 0.5:
                    solution[(couriers[i], routes[j])] = True
                else:
                    solution[(couriers[i], routes[j])] = False
        return solution
    else:
        print('No optimal solution found.')
        return None

solution = maximize_shift_filling(couriers, routes, preference_matrix, fill_difficulty_score)

if solution:
    for (courier, route), invited in solution.items():
        if invited:
            print(f'{courier} is invited to {route}')

if solution:
    # Create a list of dictionaries to store the data
    data = []
    for (courier, route), invited in solution.items():
        if invited:
            data.append({'courier_uid': courier, 'route_uid': route})

    # Create a DataFrame from the list of dictionaries
    df_solution = pd.DataFrame(data)

    # Download the DataFrame as a CSV file
    from google.colab import files
    df_solution.to_csv('shift_assignments.csv', index=False)
    files.download('shift_assignments.csv')

    df_route_score = pd.DataFrame({'route_uid': routes, 'Route Score':fill_difficulty_score})
    df_route_score.to_csv('route_score.csv', index=False)
    files.download('route_score.csv')


Solution found!
00b08e94-94fe-42bb-84da-4c25360b24ea is invited to fa4435be-5fb7-4ff2-9461-49851e9a4544
016b586e-a665-4eaa-b38c-bae8c5b954a4 is invited to 0b3e7289-be8c-407b-a300-52c59fabe356
020b48fc-84b2-480e-9418-ab1720d7e913 is invited to d9c0e8c1-58ad-405f-a857-5aa19c8c702b
023cf413-a6b6-4c54-a6fd-7907ba0c933f is invited to 4411ff82-e1df-421b-b9cb-8af8bf79c162
024523d0-fbb9-4d7e-830e-5cbab58b555e is invited to b440cddd-8083-401c-ac4b-f4590e06e7fb
029677f0-07a6-40d4-ac60-8a2f0c8a7b06 is invited to 6bc185fe-64ed-49ce-ab26-6bee57ea2159
032ad2b4-be0e-4cf9-9a6c-651262e1d1eb is invited to 76d2c1dd-35ff-4ce2-86e7-d2056bb6734a
06a38d8e-eec6-45dd-ab32-a76ffd8bb544 is invited to f362d41e-34e2-4a3f-93f1-f1894fb460c1
06c1fb1c-c4d4-4a6c-b68b-621ee07ebe65 is invited to 86b17a01-5640-418f-8315-b65ecc29d674
0766fa06-62bf-4d7a-9a98-1c6e615a85e7 is invited to 0699cb03-b7b8-4a9b-a90a-93decadff121
0793acd3-a84c-4ab9-a671-c57ae0532349 is invited to eeba5973-62ec-4073-b044-8406786f938b
0a433519-d387-4b

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
# Merge the solution dataframe with open_route_combinations and create a new column that flags which rows have a row in the solution.  Call this new dataframe all_potential_invites_with_solution and download as csv. do not use the _merge column

# Merge open_route_combinations with the solution dataframe
all_potential_invites_with_solution = pd.merge(open_courier_route_combinations, df_solution, on=['courier_uid', 'route_uid'], how='left', indicator=True)

# Create a new column that flags rows that have a corresponding row in the solution
all_potential_invites_with_solution['has_solution'] = all_potential_invites_with_solution['_merge'] == 'both'

# Remove the _merge column
all_potential_invites_with_solution = all_potential_invites_with_solution.drop('_merge', axis=1)

all_potential_invites_with_solution = pd.merge(all_potential_invites_with_solution, df_route_score, on='route_uid', how='left')


# Download the dataframe as a csv
all_potential_invites_with_solution.to_csv('all_potential_invites_with_solution.csv', index=False)
files.download('all_potential_invites_with_solution.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
# prompt: plot a map of each row with the courier lat lon and pitstop lat lon with lines between them

import folium
from folium.plugins import MarkerCluster
import pandas as pd

# Assuming your dataframe is named 'all_potential_invites_with_solution'

# Create a map centered on a specific location (adjust as needed)
map_center = [51.5074, 0.1278]  # Example: London
m = folium.Map(location=map_center, zoom_start=10)

# Create a marker cluster for better visualization of multiple markers
marker_cluster = MarkerCluster().add_to(m)

df = all_potential_invites_with_solution.loc[all_potential_invites_with_solution['has_solution']]

# Iterate through rows of the dataframe
for index, row in df.iterrows():
  if not pd.isnull(row['lat']) and not pd.isnull(row['lon']) and not pd.isnull(row['pitstop_lat']) and not pd.isnull(row['pitstop_lon']):
    # Courier location marker
    courier_marker = folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"Courier: {row['courier_uid']}"
    )
    courier_marker.add_to(marker_cluster)

    # Pitstop location marker
    pitstop_marker = folium.Marker(
        location=[row['pitstop_lat'], row['pitstop_lon']],
        popup=f"Pitstop: {row['route_uid']}"
    )
    pitstop_marker.add_to(marker_cluster)

    # Draw a line between courier and pitstop
    folium.PolyLine(
        locations=[[row['lat'], row['lon']], [row['pitstop_lat'], row['pitstop_lon']]],
        color='blue',
        weight=2,
        opacity=0.7
    ).add_to(m)

# Display the map
m